# XML to Database

In [8]:
import xml.etree.ElementTree as ET
import sqlite3

In [9]:
conn=sqlite3.connect('trackdb.sqlite')
cur=conn.cursor()

In [10]:
cur.execute(''' DROP TABLE IF EXISTS Artist ''')
cur.execute(''' DROP TABLE IF EXISTS Album; ''')
cur.execute(''' DROP TABLE IF EXISTS Track; ''')
cur.execute(''' DROP TABLE IF EXISTS GENRE; ''')
            
cur.execute('''
CREATE TABLE IF NOT EXISTS Artist(
id INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT UNIQUE,
name TEXT UNIQUE
)''')
cur.execute('''
CREATE TABLE IF NOT EXISTS Album(
id INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT UNIQUE,
artist_id INTEGER,
title TEXT UNIQUE
)''')
cur.execute('''
CREATE TABLE IF NOT EXISTS Track(
id INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT UNIQUE,
title TEXT UNIQUE,
album_id INTEGER,
len INTEGER, rating INTEGER, count INTEGER)''')

In [11]:
fname=raw_input("Enter a file name: ")
if(len(fname)<1) : fname='Library.xml'

def lookup(d, key):
    found=False
    for child in d:
        if found: return child.text
        if child.tag=='key' and child.text==key:
            found = True
    return None
stuff=ET.parse(fname)
all=stuff.findall('dict/dict/dict')
print 'Dict count:', len(all)
for entry in all:
    if (lookup(entry, 'Track ID') is None): continue
    
    name=lookup(entry, 'Name')
    artist=lookup(entry, 'Artist')
    album=lookup(entry, 'Album')
    count=lookup(entry, 'Play Count')
    rating=lookup(entry, 'Rating')
    length=lookup(entry, 'Total Time')
    
    if name is None or artist is None or album is None:
        continue
    
    #print name, artist, album, count, rating, length
    
    cur.execute('''INSERT OR IGNORE INTO Artist (name)
    VALUES ( ? )''', ( artist,  ))
    cur.execute('SELECT id FROM Artist WHERE name= ? ', (artist, ))
    artist_id=cur.fetchone()[0]
    
    cur.execute('''INSERT OR IGNORE INTO Album (title, artist_id)
    VALUES(?, ?)''', (album, artist_id))
    cur.execute('SELECT id FROM Album WHERE title=?', (album, ))
    album_id=cur.fetchone()[0]
    
    cur.execute('''INSERT OR REPLACE INTO Track
    (title, album_id, len, rating, count)
    VALUES(?,?,?,?,?)''', (name, album_id, length, rating, count))
    
    conn.commit()

Enter a file name: 
Dict count: 404


## Assignment: Multitable Database Tracks

In [12]:
import xml.etree.ElementTree as ET
import sqlite3

conn = sqlite3.connect('trackdb_assignment.sqlite')
cur = conn.cursor()

In [13]:
# Make some fresh tables using executescript()
cur.executescript('''
DROP TABLE IF EXISTS Artist;
DROP TABLE IF EXISTS Album;
DROP TABLE IF EXISTS Track;
DROP TABLE IF EXISTS Genre;

CREATE TABLE Artist (
    id  INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT UNIQUE,
    name    TEXT UNIQUE
);

CREATE TABLE Genre (
    id  INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT UNIQUE,
    name    TEXT UNIQUE
);

CREATE TABLE Album (
    id  INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT UNIQUE,
    artist_id  INTEGER,
    title   TEXT UNIQUE
);

CREATE TABLE Track (
    id  INTEGER NOT NULL PRIMARY KEY 
        AUTOINCREMENT UNIQUE,
    title TEXT  UNIQUE,
    album_id  INTEGER,
    genre_id  INTEGER,
    len INTEGER, rating INTEGER, count INTEGER
);
''')

In [14]:
fname=raw_input("Enter a file name: ")
if(len(fname)<1) : fname='Library.xml'

def lookup(d, key):
    found=False
    for child in d:
        if found: return child.text
        if child.tag=='key' and child.text==key:
            found = True
    return None
stuff=ET.parse(fname)
all=stuff.findall('dict/dict/dict')
print 'Dict count:', len(all)
for entry in all:
    if (lookup(entry, 'Track ID') is None): continue
    
    name=lookup(entry, 'Name')
    artist=lookup(entry, 'Artist')
    album=lookup(entry, 'Album')
    count=lookup(entry, 'Play Count')
    rating=lookup(entry, 'Rating')
    length=lookup(entry, 'Total Time')
    genre=lookup(entry, 'Genre')
    
    if name is None or artist is None or album is None or genre is None:
        continue
    
    print name, artist, album, count, rating, length, genre
    
    cur.execute('''INSERT OR IGNORE INTO Artist (name)
    VALUES ( ? )''', ( artist,  ))
    cur.execute('SELECT id FROM Artist WHERE name= ? ', (artist, ))
    artist_id=cur.fetchone()[0]
    
    cur.execute('''INSERT OR IGNORE INTO Genre (name)
    VALUES ( ? )''', ( genre,  ))
    cur.execute('SELECT id FROM Genre WHERE name= ? ', (genre, ))
    genre_id=cur.fetchone()[0]
    
    cur.execute('''INSERT OR IGNORE INTO Album (title, artist_id)
    VALUES(?, ?)''', (album, artist_id))
    cur.execute('SELECT id FROM Album WHERE title=?', (album, ))
    album_id=cur.fetchone()[0]
    
    cur.execute('''INSERT OR REPLACE INTO Track
    (title, album_id,genre_id, len, rating, count)
    VALUES(?,?,?,?,?,?)''', (name, album_id,genre_id, length, rating, count))
    
    conn.commit()

Enter a file name: 
Dict count: 404
Another One Bites The Dust Queen Greatest Hits 55 100 217103 Rock
Asche Zu Asche Rammstein Herzeleid 79 100 231810 Industrial
Beauty School Dropout Various Grease 48 100 239960 Soundtrack
Black Dog Led Zeppelin IV 109 100 296620 Rock
Bring The Boys Back Home Pink Floyd The Wall [Disc 2] 33 100 87118 Rock
Circles Bryan Lee Blues Is 54 60 355369 Funk
Comfortably Numb Pink Floyd The Wall [Disc 2] 36 100 384130 Rock
Crazy Little Thing Called Love Queen Greatest Hits 38 100 163631 Rock
Electric Funeral Black Sabbath Paranoid 44 100 293015 Metal
Fat Bottomed Girls Queen Greatest Hits 38 100 257515 Rock
For Those About To Rock (We Salute You) AC/DC Who Made Who 84 100 353750 Rock
Four Sticks Led Zeppelin IV 84 100 284421 Rock
Furious Angels Rob Dougan The Matrix Reloaded 54 100 330004 Soundtrack
Gelle Bryan Lee Blues Is 45 60 199836 Blues/R&B
Going To California Led Zeppelin IV 100 100 215666 Rock
Grease Various Grease 42 100 205792 Soundtrack
Hand of Doom 

In [41]:
cur.execute('''
SELECT Track.title, Artist.name, Album.title, Genre.name 
    FROM Track JOIN Genre JOIN Album JOIN Artist 
    ON Track.genre_id = Genre.ID and Track.album_id = Album.id 
        AND Album.artist_id = Artist.id
    ORDER BY Artist.name, Track.title LIMIT 3
''')
query= cur.fetchall()

In [42]:
for i in query:
    print i

(u'Chase the Ace', u'AC/DC', u'Who Made Who', u'Rock')
(u'D.T.', u'AC/DC', u'Who Made Who', u'Rock')
(u'For Those About To Rock (We Salute You)', u'AC/DC', u'Who Made Who', u'Rock')
